In [5]:
import sys
sys.path.append("../qiskit")
sys.path.append("../pytorch")
from qiskit_library import *
# from mnist import *
import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm
import torch
import numpy as np
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from lib_model_summary import summary
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import numpy as np 
import math
import os
import shutil
import os
import time
import sys
from pathlib import Path
import functools
from QC_SELF import Net
from collections import Counter


from qiskit import IBMQ
# IBMQ.delete_accounts()
IBMQ.save_account('62d0e14364f490e45b5b5e0f6eebdbc083270ffffb660c7054219b15c7ce99ab4aa3b321309c0a9d0c3fc20086baece1376297dcdb67c7b715f9de1e4fa79efb')
IBMQ.load_account()



def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits



def fire_ibmq(circuit,shots,iter,Simulation = False, printable=True,backend_name='ibmq_essex'):
    if printable:
        print(circuit)
    
    count_set = []
    start = time.time()
    for it in range(iter):
        if not Simulation:
            provider = IBMQ.get_provider('ibm-q-academic')
            # ibm-q-academic backends: 
            #  5 qubits: ibmq_valencia
            # 20 qubits: ibmq_poughkeepsie, ibmq_johannesburg,ibmq_boeblingen, ibmq_20_tokyo
            # 53 qubits: ibmq_rochester
            
            # To get a specific qubit backend: 
            backend = provider.get_backend(backend_name)
        else:
            backend = Aer.get_backend('qasm_simulator')
        job_ibm_q = execute(circuit, backend, shots=shots)
        job_monitor(job_ibm_q)
        result_ibm_q = job_ibm_q.result()
        counts = result_ibm_q.get_counts()
        count_set.append(counts)
    end = time.time()
    print("Simulation time:", end - start)

    return count_set


/home/weiwen/.local/lib/python3.6/site-packages/qiskit/providers/ibmq/credentials/configrc.py:130: UserWarning: Credentials already present. Set overwrite=True to overwrite.
  warnings.warn('Credentials already present. '
/home/weiwen/.local/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


In [6]:
q = qk.QuantumRegister(12,"q")
c = qk.ClassicalRegister(1,"c")

circuit = qk.QuantumCircuit(q, c)


circuit.ry(0.7954,q[0])
circuit.ry(2.4981,q[1])
circuit.ry(2.3766,q[3])
circuit.ry(1.7236,q[5])
circuit.ry(0.7954,q[6])
circuit.ry(2.4981,q[7])
circuit.ry(0.8371,q[9])
circuit.ry(2.5933,q[10])

circuit.x(q[3])
circuit.x(q[10])
circuit.barrier(q[3],q[11],q[10],q[9],q[8],q[7],q[6],q[5],q[4],q[1],q[0],q[2])
circuit.h(q[1])
circuit.h(q[7])
circuit.cz(q[0],q[1])
circuit.cz(q[6],q[7])
circuit.z(q[1])
circuit.h(q[7])
circuit.h(q[1])
circuit.x(q[7])
circuit.x(q[1])
circuit.cx(q[7],q[8])
circuit.cx(q[1],q[2])
circuit.ccx(q[7],q[9],q[8])
circuit.x(q[1])
circuit.ccx(q[1],q[3],q[2])
circuit.cx(q[2],q[4])
circuit.ccx(q[2],q[5],q[4])
circuit.barrier(q[3],q[11],q[10],q[9],q[8],q[7],q[6],q[5],q[4],q[1],q[0],q[2])
circuit.h(q[8])
circuit.cz(q[4],q[8])
circuit.h(q[8])
circuit.x(q[8])
circuit.cx(q[8],q[11])
circuit.x(q[8])
circuit.ccx(q[8],q[10],q[11])
circuit.barrier(q[3],q[11],q[10],q[9],q[8],q[7],q[6],q[5],q[4],q[1],q[0],q[2])
circuit.measure(q[11],c[0])



print(circuit)





         ┌────────────┐      ░                                                 »
 q_0: |0>┤ Ry(0.7954) ├──────░───────■─────────────────────────────────────────»
         ├────────────┤      ░ ┌───┐ │ ┌───┐┌───┐┌───┐     ┌───┐               »
 q_1: |0>┤ Ry(2.4981) ├──────░─┤ H ├─■─┤ Z ├┤ H ├┤ X ├──■──┤ X ├──■────────────»
         └────────────┘      ░ └───┘   └───┘└───┘└───┘┌─┴─┐└───┘┌─┴─┐          »
 q_2: |0>────────────────────░────────────────────────┤ X ├─────┤ X ├──■────■──»
         ┌────────────┐┌───┐ ░                        └───┘     └─┬─┘  │    │  »
 q_3: |0>┤ Ry(2.3766) ├┤ X ├─░────────────────────────────────────■────┼────┼──»
         └────────────┘└───┘ ░                                       ┌─┴─┐┌─┴─┐»
 q_4: |0>────────────────────░───────────────────────────────────────┤ X ├┤ X ├»
         ┌────────────┐      ░                                       └───┘└─┬─┘»
 q_5: |0>┤ Ry(1.7236) ├──────░──────────────────────────────────────────────■──»
         ├────────────┤     

In [7]:


qc_shots = 100000
num_c_reg = 2

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,True,False)
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)


Start simulation:
Job Status: job has successfully run
Simulation time: 5.073574066162109
0 0.62952
From QC: [{'1': 62952, '0': 37048}]
Simulation elasped time: 5.076713800430298


In [ ]:
qc_shots = 8192
print("="*50)
print("Start run:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,False,False,backend_name="ibmq_cambridge")
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)


sys.exit(0)

Start run:
Job Status: job is actively running 

In [ ]:

def generate_circuit(model,input,num_layer,arch,opt=0):
    print("="*100)
    print("Generating Quantum Circuit from Machine Learning Model")
    print("="*100)
    
    in_size = input.shape[0]
    print(in_size)
    
    for name,para in model.named_parameters():
        print(name,torch.tensor(para))

    
    qcircuit = {}
    q_in_whole = {}
    q_aux_whole = {}
    q_out = {}
    for i in range(num_layer):
        for sub_qc in range(arch[i]):
            q_in = qk.QuantumRegister(in_size,"io"+"-"+str(i)+"-"+str(sub_qc))
            q_in_whole[i,sub_qc]=q_in
            w = model.state_dict()["fc"+str(i+1)+".weight"][sub_qc]
            w = (w>0).int()            
            
            if model.state_dict()["qc"+str(i+1)+"a.x_l_0_5"][sub_qc]==0:
                q_aux =  qk.QuantumRegister(2,"aux"+"-"+str(i)+"-"+str(sub_qc))
                q_aux_whole[i,sub_qc] = q_aux
                # print("qc"+str(i)+str(sub_qc))
                qcircuit["qc"+str(i)+str(sub_qc)] = qk.QuantumCircuit(q_in, q_aux)
                circuit = qcircuit["qc"+str(i)+str(sub_qc)]                
                if i==0:
                    init(circuit, input, q_in)
                circuit.h(q_in[1])
                circuit.cz(q_in[0],q_in[1])
                if w[0]!=w[1]:
                    circuit.z(q_in[1])
                circuit.h(q_in[1])
                circuit.x(q_in[1])
                circuit.cx(q_in[1],q_aux[0])                
                mul = model.state_dict()["qc"+str(i+1)+"a.x_running_rot"][sub_qc]
                if i!=num_layer-1:
                    mul = mul*model.state_dict()["qc"+str(i+1)+".x_running_rot"][sub_qc]            
                init(circuit, mul.view(1)*2-1, q_aux[1])                
                circuit.ccx(q_in[1],q_aux[1],q_aux[0])
                q_out[i,sub_qc] = q_aux[0]
                # print(qcircuit["qc"+str(i)+str(sub_qc)])
            elif model.state_dict()["qc"+str(i+1)+"a.x_l_0_5"][sub_qc]==1 and i!=num_layer-1:
                q_aux =  qk.QuantumRegister(4,"aux"+"-"+str(i)+"-"+str(sub_qc))
                q_aux_whole[i,sub_qc] = q_aux
                # print("qc"+str(i)+str(sub_qc))
                qcircuit["qc"+str(i)+str(sub_qc)] = qk.QuantumCircuit(q_in, q_aux)
                circuit = qcircuit["qc"+str(i)+str(sub_qc)]                
                if i==0:
                    init(circuit, input, q_in)
                circuit.h(q_in[1])
                circuit.cz(q_in[0],q_in[1])
                if w[0]!=w[1]:
                    circuit.z(q_in[1])
                circuit.h(q_in[1])
                circuit.x(q_in[1])
                circuit.cx(q_in[1],q_aux[0]) 


                mul = model.state_dict()["qc"+str(i+1)+"a.x_running_rot"][sub_qc]                            
                init(circuit, mul.view(1)*2-1, q_aux[1])                
                circuit.x(q_in[1])
                circuit.x(q_aux[1])
                circuit.ccx(q_in[1],q_aux[1],q_aux[0])
                
                circuit.cx(q_aux[0],q_aux[2])
                
                mul = model.state_dict()["qc"+str(i+1)+".x_running_rot"][sub_qc]
                init(circuit, mul.view(1)*2-1, q_aux[3])
                circuit.ccx(q_aux[0],q_aux[3],q_aux[2])
                q_out[i,sub_qc] = q_aux[2]                
                # print(qcircuit["qc"+str(i)+str(sub_qc)])
        
            else:
                q_aux =  qk.QuantumRegister(2,"aux"+"-"+str(i)+"-"+str(sub_qc))
                q_aux_whole[i,sub_qc] = q_aux
                # print("qc"+str(i)+str(sub_qc))
                qcircuit["qc"+str(i)+str(sub_qc)] = qk.QuantumCircuit(q_in, q_aux)
                circuit = qcircuit["qc"+str(i)+str(sub_qc)]                
                if i==0:
                    init(circuit, input, q_in)
                circuit.h(q_in[1])
                circuit.cz(q_in[0],q_in[1])
                if w[0]!=w[1]:
                    circuit.z(q_in[1])
                circuit.h(q_in[1])
                circuit.x(q_in[1])
                circuit.cx(q_in[1],q_aux[0])                
                mul = model.state_dict()["qc"+str(i+1)+"a.x_running_rot"][sub_qc]                            
                init(circuit, mul.view(1)*2-1, q_aux[1])                
                circuit.x(q_in[1])
                circuit.x(q_aux[1])
                circuit.ccx(q_in[1],q_aux[1],q_aux[0])
                # print(qcircuit["qc"+str(i)+str(sub_qc)])
                q_out[i,sub_qc] = q_aux[0]
        
        
    qc3 = qcircuit["qc00"]+qcircuit["qc01"]
    qc3.barrier()
    
    qc3.cx(q_out[(0,0)],q_out[(0,1)])
        
    point = len(qc3.qasm())
    
    qc3 += qcircuit["qc10"]
    
    
    start = len(qc3.qasm())
    qc3.cx(q_out[(0,1)],q_in_whole[(1,0)][0])
    end = len(qc3.qasm())-1
    
    # print("===")
    # qc3.qasm().insert(qc3.qasm()[start:end],point)
    # print("===")
        
    
    qc3.qasm = qc3.qasm()[:point]+qc3.qasm()[start:end]+qc3.qasm()[point:start]
    
    
    # qc3.insert()
    
    
    
    
    
    
    print(qc3)
    # q_io = qk.QuantumRegister(2,"reg") 
    # q1 = qk.QuantumCircuit(q_io)
    # print("="*100)
    # qc3 = qcircuit["qc00"]+q1
    # print(qc3)
    # top_circuit = qcircuit["qc00"] 
    # top_circuit.combine(qcircuit["qc01"])
    # # top_circuit = qcircuit["qc00"] + qcircuit["qc01"]
    # print(top_circuit)
    sys.exit(0)    
    
generate_circuit(model,torch.tensor([0.95,0.95])*2-1,2,[2,2])